# Visualization Utilities

In [1]:
from IPython.display import display, HTML, Image
from ipywidgets.widgets import FloatSlider, IntSlider, Dropdown, SelectMultiple, Button
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score
import seaborn as sns

# Interface to select classification target and feature set

In [3]:
def interface(df, target, label):
    all_columns = df.columns.values.tolist()
    target_widget = Dropdown(
        margin=10,
        options=all_columns,
        value=target,
        description='Target Variable:',
    )
    features_widget = SelectMultiple(
        margin=10,
        description="Input Features:",
        options=all_columns
    )
    button = Button(description=label, width=100)
    return target_widget, features_widget, button

def explore_matrix(dependent_variable, input_features):
    if dependent_variable in input_features:
        raise Exception('Dependent Variable should not be one of the input features.')
    pair_columns = [dependent_variable] + list(input_features)
    sns.pairplot(df[pair_columns], hue=dependent_variable)

# Display Change in size of the dataset after dropping rows with missing values

In [4]:
def size_after_droppping_rows_with_missing_values(count_before_drop, count_after_drop):
    print("""
    Total data points before dropna: \t{0}
    Total data points after dropna: \t{1}
    Data points with missing values: \t{2}
    """.format(count_before_drop, count_after_drop, count_before_drop - count_after_drop))

# Display characteristics of train, validation, and test datasets

In [5]:
def balance_as_text(label, label_ratio, data_size, numerator, denominator):
    template = """
{0}
   Size: {2}
   {1} {3:.3f}
""".format(label, label_ratio, data_size, numerator / (1.0 * denominator))
    return template

def display_partitioning(y_train, y_validation,  y_test, positive_class, negative_class):
    print(balance_as_text("Training dataset", "positive/negative ratio:", 
                          len(y_train), 
                          y_train.values.tolist().count(positive_class),
                          y_train.values.tolist().count(negative_class)))
    print(balance_as_text("Validation dataset", "positive/negative ratio:",
                          len(y_validation), 
                          y_validation.values.tolist().count(positive_class),
                          y_validation.values.tolist().count(negative_class)))
    print(balance_as_text("Test dataset", "positive/negative ratio:",
                          len(y_test), 
                          y_test.values.tolist().count(positive_class),
                          y_test.values.tolist().count(negative_class)))    

# Compute and show evaluation metrics for logistic regression

In [6]:
def plot_class_separation(y_negative, y_positive, num_bins, threshold_value):

    n0, bins0, patches0 = plt.hist(y_negative, num_bins,  facecolor='red', alpha=0.3, label=negative_class)
    n1, bins1, patches1 = plt.hist(y_positive, num_bins,  facecolor='blue', alpha=0.3, label=positive_class)
    
    plt.axvline(x=threshold_value, ymin=0, ymax=1, linewidth=2, color='#aaaaaa')

    plt.xlabel('probability')
    plt.ylabel('total')
    plt.title('Class separation')
    plt.grid(True)
    plt.show()


In [7]:
def show_evaluation_metrics_logistics(model, threshold_value, x, y): 
          
    y_pred = apply_threshold(model, threshold_value, x)
    report = classification_report(y, y_pred, target_names=class_labels)
    cm = confusion_matrix(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
  
    tpr_value = cm[1][1]/(1.0*y.values.tolist().count(1))
    fpr_value = cm[0][1]/(1.0*y.values.tolist().count(0))

    predicted_positive_count = y_pred.count(1)
    predicted_negative_count = y_pred.count(0)
        
    display(HTML("""
    <div>
      <table>
        <tr> <th>Accuracy (Evaluation):</th> <td colspan="6" style="background-color:#cceecc"> {ACCURACY:.3f} </td></tr>
        <tr> <th> {CTRL_LABEL} </th> <td> {Threshold:.4f}</td> <th>&nbsp;</th> <th>&nbsp;</th> <th>Predicted {POSITIVE_CLASS}</th> <th>Predicted {NEGATIVE_CLASS}</th> <th> Total </th></tr>
        <tr> <th> True Positive Rate </th> <td style="background-color:#eeffcc">{TPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {POSITIVE_CLASS}</th> <td style="background-color:#ccffcc">{TP}</td> <td style="background-color:#eedddd">{FN}</td> 
             <td style="background-color:#ccffff"> {Y_TEST_POSITIVE} </td>
        </tr>

        <tr> <th> False positive Rate </th> <td style="background-color:#ddeebb">{FPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {NEGATIVE_CLASS}</th> <td style="background-color:#ffdddd">{FP}</td> <td style="background-color:#bbeebb">{TN}</td> 
             <td style="background-color:#aadddd"> {Y_TEST_NEGATIVE} </td>
        </tr>
        <tr>  <th colspan="3">&nbsp;</th> <th>Total</th> <td style="background-color:#ffffcc">{PREDICTED_POSITIVE}</td> <td style="background-color:#ddddaa">{PREDICTED_NEGATIVE}</td> <td style="background-color:#dddddd">{TOTAL}</td> </tr>

        <tr> <td colspan="6"><pre>{REPORT}</pre></td>   </tr>
        
      </table>
    </div>

    """.format(Threshold=threshold_value, 
                     CTRL_LABEL='Threshold', 
                     TPR=tpr_value, 
                     FPR=fpr_value, 
                     REPORT=report,  
                     ACCURACY=accuracy,
                     POSITIVE_CLASS=positive_class,
                     NEGATIVE_CLASS=negative_class,
                     Y_TEST_POSITIVE=y.values.tolist().count(1),
                     Y_TEST_NEGATIVE=y.values.tolist().count(0),
                     PREDICTED_POSITIVE=predicted_positive_count,
                     PREDICTED_NEGATIVE=predicted_negative_count,
                     TOTAL=predicted_positive_count + predicted_negative_count,
                     TN=cm[0][0],
                     FN=cm[1][0],
                     FP=cm[0][1],
                     TP=cm[1][1])))
   

# Compute and show evaluation metrics for other models

In [8]:
def show_evaluation_metrics(clf, control_label, k, x, y): 
    
    y_pred = clf.predict(x)
    report = classification_report(y, y_pred, target_names=class_labels)
    cm = confusion_matrix(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    
    tpr_value = cm[0][0]/(1.0*y.values.tolist().count(positive_class))
    fpr_value = cm[1][0]/(1.0*y.values.tolist().count(negative_class))
    
    predicted_positive_count = y_pred.tolist().count(positive_class)
    predicted_negative_count = y_pred.tolist().count(negative_class)
    
    display(HTML("""
    <div>
      <table>
        <tr> <th>Accuracy (Evaluation):</th> <td colspan="5" style="background-color:#cceecc"> {ACCURACY:.3f} </td></tr>
        <tr> <th> {CTRL_LABEL} </th> <td> {Complexity:.1f} </td> <th>&nbsp;</th> <th>&nbsp;</th> <th>Predicted {POSITIVE_CLASS}</th> <th>Predicted {NEGATIVE_CLASS}</th> <th> Total </th></tr>
        <tr> <th> True Positive Rate </th> <td style="background-color:#eeffcc">{TPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {POSITIVE_CLASS}</th> <td style="background-color:#ccffcc">{TP}</td> <td style="background-color:#eedddd">{FN}</td> 
             <td style="background-color:#ccffff"> {Y_TEST_POSITIVE} </td>
        </tr>

        <tr> <th> False positive Rate </th> <td style="background-color:#ddeebb">{FPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {NEGATIVE_CLASS}</th> <td style="background-color:#ffdddd">{FP}</td> <td style="background-color:#bbeebb">{TN}</td> 
             <td style="background-color:#aadddd"> {Y_TEST_NEGATIVE} </td>
        </tr>
        <tr>  <th colspan="3">&nbsp;</th> <th>Total</th> <td style="background-color:#ffffcc">{PREDICTED_POSITIVE}</td> <td style="background-color:#ddddaa">{PREDICTED_NEGATIVE}</td> <td style="background-color:#dddddd">{TOTAL}</td> </tr>

        <tr> <td colspan="6"><pre>{REPORT}</pre></td>   </tr>
        
      </table>
    </div>

    """.format(Complexity=k,
              CTRL_LABEL= control_label, 
              TPR=tpr_value, 
              FPR=fpr_value, 
              REPORT=report,  
              ACCURACY=accuracy,
              POSITIVE_CLASS=positive_class,
              NEGATIVE_CLASS=negative_class,
              Y_TEST_POSITIVE=y.values.tolist().count(positive_class),
              Y_TEST_NEGATIVE=y.values.tolist().count(negative_class),
              PREDICTED_POSITIVE=predicted_positive_count,
              PREDICTED_NEGATIVE=predicted_negative_count,
              TOTAL=predicted_positive_count + predicted_negative_count,
              TN=cm[1][1],
              FN=cm[0][1],
              FP=cm[1][0],
              TP=cm[0][0])))

# Plot Interactive ROC/Error Curve with computed metrics

In [9]:
def plot_roc(fpr, tpr, fpr_t=None, tpr_t=None, figsize=(10,9)):
    plt.style.use('ggplot')
    plt.figure(figsize=figsize)
    plt.title('Receiver operating characteristic curve (ROC curve)')
    plt.plot(fpr, tpr, lw=1, label ='Logistic Regression')
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
    if fpr_t is not None:
        plt.axvline(x=fpr_t, ymin=0, ymax=1, linewidth=1, color='#aaaaaa')
    if tpr_t is not None:
        plt.axhline(y=tpr_t, xmin=0, xmax=1, linewidth=1, color='#aaaaaa')
    plt.legend(loc="lower right")
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

def apply_threshold(model, threshold, x):
    y_predicted_proba = model.predict_proba(x)
    y_1 = [int(c1 > threshold) for _, c1 in y_predicted_proba]
    return y_1


def plot_error(depths, error_validation, error_train, model_complexity):
    
    plt.style.use('ggplot')
    plt.figure(figsize=(10,9))
    plt.title('Validation')
    plt.plot(depths, error_validation, lw=1, label ='Validation dataset')
    plt.plot(depths, error_train, lw=1, label ='Train dataset')
    plt.axvline(x=model_complexity, ymin=0, ymax=1, linewidth=1, color='#aaaaaa')
    plt.legend(loc="lower right")
    plt.xlabel('Model Complexity')
    plt.ylabel('Error Rate')
    plt.ylim([-0.05, 1.05])    
    plt.show()    
       
    
def evaluate_error(model_map, control_label, display_tree, model_complexity):
    
    y_pred = predicted_map[model_complexity]
    report = classification_report(y_validation, y_pred, target_names=class_labels)
    cm = confusion_matrix(y_validation, y_pred)
    accuracy = accuracy_score(y_validation, y_pred)

    tpr_value = cm[0][0]/(1.0*y_validation.values.tolist().count(positive_class))
    fpr_value = cm[1][0]/(1.0*y_validation.values.tolist().count(negative_class))
    
    plot_error(complexities, error_validation, error_train, model_complexity)    
      
    display(HTML("""
    <div>
      <table>
        <tr> <th>Accuracy (Evaluation):</th> <td colspan="5" style="background-color:#cceecc"> {ACCURACY:.3f} </td></tr>

        <tr> <th> {CTRL_LABEL} </th> <td>{Complexity:.1f} </td> <th>&nbsp;</th> <th>&nbsp;</th> <th>Predicted {POSITIVE_CLASS}</th> <th>Predicted {NEGATIVE_CLASS}</th> <th> Total </th></tr>
        <tr> <th> True Positive Rate </th> <td style="background-color:#eeffcc">{TPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {POSITIVE_CLASS}</th> <td style="background-color:#ccffcc">{TP}</td> <td style="background-color:#eedddd">{FN}</td> 
             <td style="background-color:#ccffff"> {Y_TEST_POSITIVE} </td>
        </tr>

        <tr> <th> False Positive Rate </th> <td style="background-color:#ddeebb">{FPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {NEGATIVE_CLASS}</th> <td style="background-color:#ffdddd">{FP}</td> <td style="background-color:#bbeebb">{TN}</td> 
             <td style="background-color:#aadddd"> {Y_TEST_NEGATIVE} </td>
        </tr>
        <tr>  <th colspan="3">&nbsp;</th> <th>Total</th> <td style="background-color:#ffffcc">{PREDICTED_POSITIVE}</td> <td style="background-color:#ddddaa">{PREDICTED_NEGATIVE}</td> </tr>

        <tr> <td colspan="6"><pre>{REPORT}</pre></td>   </tr>
        
      </table>
    </div>

    """.format(      Complexity=model_complexity,
                     CTRL_LABEL= control_label, 
                     TPR=tpr_value, 
                     FPR=fpr_value, 
                     REPORT=report,  
                     ACCURACY=accuracy,
                     POSITIVE_CLASS=positive_class,
                     NEGATIVE_CLASS=negative_class,
                     Y_TEST_POSITIVE=y_validation.values.tolist().count(positive_class),
                     Y_TEST_NEGATIVE=y_validation.values.tolist().count(negative_class),
                     PREDICTED_POSITIVE=y_pred.tolist().count(positive_class),
                     PREDICTED_NEGATIVE=y_pred.tolist().count(negative_class),
                     TN=cm[1][1],
                     FN=cm[0][1],
                     FP=cm[1][0],
                     TP=cm[0][0])))
    if display_tree:
        show_tree(model_map[model_complexity], class_names)

    
def interactive_roc(model, control_label, cursor):
    threshold_value = cursor

    fpr_float, tpr_float, thresholds = roc_curve(y_validation, probs_[:, 1])

    y_predicted = apply_threshold(model, threshold_value, x_validation)
    report = classification_report(y_validation, y_predicted, target_names=class_labels)
    cm = confusion_matrix(y_validation, y_predicted)
    accuracy = accuracy_score(y_validation, y_predicted)
    fpr_threshold, tpr_threshold, _ = roc_curve(y_validation, y_predicted)    
    auc_score = auc(fpr_threshold, tpr_threshold)  
    
    tpr_value = cm[1][1]/(1.0*y_validation.values.tolist().count(1))
    fpr_value = cm[0][1]/(1.0*y_validation.values.tolist().count(0))
   
    plot_roc(fpr_float, tpr_float, fpr_value, tpr_value, figsize=(7,6.3))

    predicted_positive_count = y_predicted.count(1)
    predicted_negative_count = y_predicted.count(0)

    
    display(HTML("""
    <div>
      <table>
        <tr> <th>Accuracy (Evaluation):</th> <td  style="background-color:#cceecc"> {ACCURACY:.3f} </td>  
             <th>Area Under Curve (AUC):</th> <td  style="background-color:#cceecc" colspan="4"> {AUC:.3f} </td> 
        </tr>
        <tr> <th> {CTRL_LABEL} </th> <td> {Threshold:.4f}</td> <th>&nbsp;</th> <th>&nbsp;</th> <th>Predicted {POSITIVE_CLASS}</th> <th>Predicted {NEGATIVE_CLASS}</th> <th> Total </th></tr>
        <tr> <th> True Positive Rate </th> <td style="background-color:#eeffcc">{TPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {POSITIVE_CLASS}</th> <td style="background-color:#ccffcc">{TP}</td> <td style="background-color:#eedddd">{FN}</td> 
             <td style="background-color:#ccffff"> {Y_TEST_POSITIVE} </td>
        </tr>

        <tr> <th> False Positive Rate </th> <td style="background-color:#ddeebb">{FPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {NEGATIVE_CLASS}</th> <td style="background-color:#ffdddd">{FP}</td> <td style="background-color:#bbeebb">{TN}</td> 
             <td style="background-color:#aadddd"> {Y_TEST_NEGATIVE} </td>
        </tr>
        <tr>  <th colspan="3">&nbsp;</th> <th>Total</th> <td style="background-color:#ffffcc">{PREDICTED_POSITIVE}</td> <td style="background-color:#ddddaa">{PREDICTED_NEGATIVE}</td>  <td style="background-color:#dddddd">{TOTAL}</td> </tr>

        <tr> <td colspan="6"><pre>{REPORT}</pre></td>   </tr>
        
      </table>
    </div>

    """.format(Threshold=threshold_value, 
                 CTRL_LABEL=control_label, 
                 TPR=tpr_value, 
                 FPR=fpr_value, 
                 REPORT=report,  
                 ACCURACY=accuracy,
                 AUC=auc_score,
                 POSITIVE_CLASS=positive_class,
                 NEGATIVE_CLASS=negative_class,
                 Y_TEST_POSITIVE=y_validation.values.tolist().count(1),
                 Y_TEST_NEGATIVE=y_validation.values.tolist().count(0),
                 PREDICTED_POSITIVE=predicted_positive_count,
                 PREDICTED_NEGATIVE=predicted_negative_count,
                 TOTAL=predicted_positive_count + predicted_negative_count,
                 TN=cm[0][0],
                 FN=cm[1][0],
                 FP=cm[0][1],
                 TP=cm[1][1])))

In [10]:
def interactive_class_separation(model, control_label, num_bins, cursor):

    threshold_value = cursor
    
    y_pred = apply_threshold(model, threshold_value, x_validation)
    report = classification_report(y_validation, y_pred, target_names=class_labels)
    cm = confusion_matrix(y_validation, y_pred)
    accuracy = accuracy_score(y_validation, y_pred)
    fpr, tpr, _ = roc_curve(y_validation, y_pred)
    auc_score = auc(fpr, tpr)
    
    tpr_value = cm[1][1]/(1.0*y_validation.values.tolist().count(1))
    fpr_value = cm[0][1]/(1.0*y_validation.values.tolist().count(0))
    
    plot_class_separation(y_0, y_1, num_bins, threshold_value)
    
    predicted_positive_count = y_pred.count(1)
    predicted_negative_count = y_pred.count(0)
    
    display(HTML("""
    <div>
      <table>
        <tr> <th>Accuracy (Evaluation):</th> <td  style="background-color:#cceecc"> {ACCURACY:.3f} </td>  
             <th>Area Under Curve (AUC):</th> <td  style="background-color:#cceecc" colspan="4"> {AUC:.3f} </td> 
        </tr>
        <tr> <th> {CTRL_LABEL} </th> <td> {Threshold:.4f}</td> <th>&nbsp;</th> <th>&nbsp;</th> <th>Predicted {POSITIVE_CLASS}</th> <th>Predicted {NEGATIVE_CLASS}</th> <th> Total </th></tr>
        <tr> <th> True Positive Rate </th> <td style="background-color:#eeffcc">{TPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {POSITIVE_CLASS}</th> <td style="background-color:#ccffcc">{TP}</td> <td style="background-color:#eedddd">{FN}</td> 
             <td style="background-color:#ccffff"> {Y_TEST_POSITIVE} </td>
        </tr>

        <tr> <th> False positive Rate </th> <td style="background-color:#ddeebb">{FPR:.3f}</td>  <th>&nbsp;</th> 
             <th>Real {NEGATIVE_CLASS}</th> <td style="background-color:#ffdddd">{FP}</td> <td style="background-color:#bbeebb">{TN}</td> 
             <td style="background-color:#aadddd"> {Y_TEST_NEGATIVE} </td>
        </tr>
        <tr>  <th colspan="3">&nbsp;</th> <th>Total</th> <td style="background-color:#ffffcc">{PREDICTED_POSITIVE}</td> <td style="background-color:#ddddaa">{PREDICTED_NEGATIVE}</td> <td style="background-color:#dddddd">{TOTAL}</td> </tr>

        <tr> <td colspan="6"><pre>{REPORT}</pre></td>   </tr>
        
      </table>
    </div>

    """.format(Threshold=threshold_value, 
                     CTRL_LABEL='Threshold', 
                     TPR=tpr_value, 
                     FPR=fpr_value, 
                     REPORT=report,  
                     ACCURACY=accuracy,
                     AUC=auc_score,
                     POSITIVE_CLASS=positive_class,
                     NEGATIVE_CLASS=negative_class,
                     TOTAL=predicted_positive_count + predicted_negative_count,
                     Y_TEST_POSITIVE=y_validation.values.tolist().count(1),
                     Y_TEST_NEGATIVE=y_validation.values.tolist().count(0),
                     PREDICTED_POSITIVE=predicted_positive_count,
                     PREDICTED_NEGATIVE=predicted_negative_count,
                     TN=cm[0][0],
                     FN=cm[1][0],
                     FP=cm[0][1],
                     TP=cm[1][1])))
    

In [11]:
def find_closest(A, target):
    idx = A.searchsorted(target)
    idx = np.clip(idx, 1, len(A)-1)
    left = A[idx-1]
    right = A[idx]
    idx -= target - left < right - target
    return idx

In [13]:
def check_confusion_matrix(model, x, y):
    y_predicted = model.predict(x)
    d = zip(y_validation, y_predicted) 
    tp = [(a,b) for (a,b) in d if (b==1 and a==1)]
    fp = [(a,b) for (a,b) in d if (b==1 and a==0) ]
    tn = [(a,b) for (a,b) in d if (b==0 and a==0) ]
    fn = [(a,b) for (a,b) in d if (b==0 and a==1) ]
    print("tp={tp} fp={fp} tn={tn} fn={fn}".format(tp=len(tp), fp=len(fp), tn=len(tn), fn=len(fn)))

In [14]:
def display_oob_error_rate(X, y, ensemble_clfs, min_estimators, max_estimators, target_n_estimators=None):
    # Author: Kian Ho <hui.kian.ho@gmail.com>
    #         Gilles Louppe <g.louppe@gmail.com>
    #         Andreas Mueller <amueller@ais.uni-bonn.de>
    #
    # License: BSD 3 Clause

    import matplotlib.pyplot as plt
    from collections import OrderedDict
    from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

    # NOTE: Setting the `warm_start` construction parameter to `True` disables
    # support for paralellised ensembles but is necessary for tracking the OOB
    # error trajectory during training.

    # Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
    error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

    # Range of `n_estimators` values to explore.
    for label, clf in ensemble_clfs:
        for i in range(min_estimators, max_estimators + 1):
            clf.set_params(n_estimators=i)
            clf.fit(X, y)

            # Record the OOB error for each `n_estimators=i` setting.
            oob_error = 1 - clf.oob_score_
            error_rate[label].append((i, oob_error))

    # Generate the "OOB error rate" vs. "n_estimators" plot.
    for label, clf_err in error_rate.items():
        xs, ys = zip(*clf_err)
        plt.plot(xs, ys, label=label)

    if target_n_estimators is not None:
        plt.axvline(x=target_n_estimators, ymin=0, ymax=1, linewidth=1, color='#aaaaaa')

    plt.xlim(min_estimators, max_estimators)
    plt.xlabel("n_estimators")
    plt.ylabel("OOB error rate")
    plt.legend(loc="upper right")
    plt.show()